# Forecasting Tire Wear Impact on Lap Time in Formula 1
## Mathematical Physics-Based Model using FastF1 Telemetry

**Objective**: Develop mathematical formulas to calculate tire wear from telemetry data and predict lap time degradation. Then simulate improved tire scenarios.

**Approach**: Physics-based formulas (NO AI/ML models) using:
- Energy dissipation calculations
- Compound-specific wear coefficients  
- Temperature and load factors
- Cumulative degradation integration
- Lap time impact functions

## 1. Setup and Imports

In [ ]:
# Install FastF1 if not already installed
# !pip install fastf1

import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Enable FastF1 cache for faster loading
cache_dir = Path('./fastf1_cache')
cache_dir.mkdir(exist_ok=True)
fastf1.Cache.enable_cache(str(cache_dir))

print("FastF1 version:", fastf1.__version__)
print("Cache enabled at:", cache_dir.absolute())

## 2. Load F1 Session Data

In [ ]:
# Configuration: Select race, session, and driver
YEAR = 2023
RACE = 'Monaco'  # Can be: 'Monaco', 'Silverstone', 'Monza', etc.
SESSION = 'R'    # 'R' = Race, 'Q' = Qualifying, 'FP1', 'FP2', 'FP3'
DRIVER = 'VER'   # Three-letter driver code

print(f"Loading {YEAR} {RACE} Grand Prix - {SESSION} session...")
session = fastf1.get_session(YEAR, RACE, SESSION)
session.load()

print(f"\nSession loaded successfully!")
print(f"Total laps in session: {len(session.laps)}")

## 3. Extract Driver Data

In [ ]:
# Get laps for selected driver
driver_laps = session.laps.pick_driver(DRIVER)

# Filter out invalid laps (pit laps, first lap, safety car, etc.)
valid_laps = driver_laps[
    (driver_laps['IsAccurate'] == True) & 
    (driver_laps['LapTime'].notna())
].copy()

print(f"Driver: {DRIVER}")
print(f"Valid laps: {len(valid_laps)}")
print(f"\nTire compounds used:")
print(valid_laps['Compound'].value_counts())

# Display first few laps
display_cols = ['LapNumber', 'LapTime', 'Compound', 'TyreLife', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST']
print("\nSample lap data:")
valid_laps[display_cols].head(10)

## 4. Extract Telemetry for Each Lap

For each lap, we'll extract detailed telemetry to calculate wear factors.

In [ ]:
def extract_lap_telemetry_features(lap):
    """
    Extract telemetry features from a single lap.
    Returns a dictionary of aggregated telemetry metrics.
    """
    try:
        telemetry = lap.get_telemetry()
        
        if telemetry.empty:
            return None
        
        # Basic statistics
        features = {
            # Speed metrics
            'speed_mean': telemetry['Speed'].mean(),
            'speed_max': telemetry['Speed'].max(),
            'speed_std': telemetry['Speed'].std(),
            
            # Throttle metrics (0-100%)
            'throttle_mean': telemetry['Throttle'].mean(),
            'throttle_max': telemetry['Throttle'].max(),
            'throttle_full_pct': (telemetry['Throttle'] > 95).mean() * 100,
            
            # Brake metrics (0-100% or binary depending on telemetry)
            'brake_mean': telemetry['Brake'].mean() if 'Brake' in telemetry.columns else 0,
            'brake_applications': (telemetry['Brake'] > 0).sum() if 'Brake' in telemetry.columns else 0,
            
            # RPM and gear
            'rpm_mean': telemetry['RPM'].mean() if 'RPM' in telemetry.columns else 0,
            'rpm_max': telemetry['RPM'].max() if 'RPM' in telemetry.columns else 0,
            'gear_mean': telemetry['nGear'].mean() if 'nGear' in telemetry.columns else 0,
            'gear_shifts': (telemetry['nGear'].diff().abs() > 0).sum() if 'nGear' in telemetry.columns else 0,
            
            # DRS usage
            'drs_active_pct': (telemetry['DRS'] > 8).mean() * 100 if 'DRS' in telemetry.columns else 0,
            
            # Time-based
            'telemetry_points': len(telemetry),
            'lap_duration': telemetry['Time'].max() - telemetry['Time'].min()
        }
        
        return features
    
    except Exception as e:
        print(f"Error extracting telemetry for lap: {e}")
        return None

# Extract features for all valid laps
print("Extracting telemetry features for each lap...")
telemetry_features = []

for idx, lap in valid_laps.iterrows():
    features = extract_lap_telemetry_features(lap)
    if features:
        features['lap_number'] = lap['LapNumber']
        features['lap_time_seconds'] = lap['LapTime'].total_seconds()
        features['compound'] = lap['Compound']
        features['tyre_life'] = lap['TyreLife']
        telemetry_features.append(features)

# Create DataFrame
df = pd.DataFrame(telemetry_features)
print(f"\nExtracted features for {len(df)} laps")
print("\nFeature summary:")
df.head()

## 5. Mathematical Tire Wear Model

### Formula Components:

#### 5.1 Energy Dissipation (Primary Wear Driver)

Tire wear is proportional to mechanical energy dissipated through the tire-road interface:

$$W_{total} = W_{braking} + W_{cornering} + W_{acceleration} + W_{thermal}$$

**Braking Energy Dissipation:**
$$W_{braking} = \sum_{i} m \cdot v_i \cdot \Delta v_i \cdot B_i$$

where:
- $m$ = vehicle mass (~800 kg)
- $v_i$ = speed at point $i$
- $\Delta v_i$ = speed reduction
- $B_i$ = brake intensity (0-1)

**Cornering Energy (Lateral Forces):**
$$W_{cornering} = \sum_{i} \frac{m \cdot v_i^2}{r_i} \cdot d_i$$

Approximated by speed variance:
$$W_{cornering} \approx k_{corner} \cdot \sigma_{speed}^2 \cdot N_{points}$$

**Acceleration Energy (Slip):**
$$W_{acceleration} = \sum_{i} T_i \cdot v_i \cdot \Delta t_i$$

where $T_i$ = throttle percentage

**Thermal Load:**
$$W_{thermal} = k_{thermal} \cdot (\bar{v} \cdot t_{lap} + N_{gear\_shifts})$$

#### 5.2 Compound-Specific Wear Coefficients

Different compounds have different base wear rates:

$$k_{compound} = \begin{cases}
1.5 & \text{SOFT (high grip, high wear)} \\
1.0 & \text{MEDIUM (balanced)} \\
0.6 & \text{HARD (low wear, lower grip)}
\end{cases}$$

#### 5.3 Temperature Effect (Exponential)

Tire wear increases exponentially with temperature:

$$T_{factor} = e^{\alpha \cdot T_{normalized}}$$

Temperature proxy from sustained high speed and brake usage:
$$T_{normalized} = 0.4 \cdot \frac{\bar{v}}{v_{max}} + 0.3 \cdot \frac{N_{brake}}{N_{max}} + 0.3 \cdot \frac{\bar{throttle}}{100}$$

#### 5.4 Total Lap Wear Index

$$W_{lap} = k_{compound} \cdot (W_{braking} + W_{cornering} + W_{acceleration} + W_{thermal}) \cdot T_{factor}$$

#### 5.5 Cumulative Degradation

$$D_{cumulative}(n) = \sum_{i=1}^{n} W_{lap,i}$$

Normalized degradation state (0 = new, 1 = worn):
$$D_{normalized}(n) = 1 - e^{-\beta \cdot D_{cumulative}(n)}$$

#### 5.6 Lap Time Impact

As tires degrade, lap time increases due to reduced grip:

$$t_{lap}(n) = t_{baseline} \cdot (1 + \gamma \cdot D_{normalized}(n))$$

where:
- $t_{baseline}$ = best lap time on fresh tires
- $\gamma$ = sensitivity factor (typically 0.05-0.15, meaning 5-15% slower on worn tires)
- $\beta$ = decay rate constant

### Improved Tire Simulation

To simulate a "better" tire, we modify the wear coefficient:

$$k_{compound,improved} = k_{compound} \cdot (1 - improvement\_factor)$$

For example, 30% improvement: $improvement\_factor = 0.30$

## 6. Implement Mathematical Formulas

In [ ]:
# Physical constants and coefficients
VEHICLE_MASS = 800  # kg (F1 car + driver + fuel approximation)

# Compound-specific wear coefficients
COMPOUND_COEFFICIENTS = {
    'SOFT': 1.5,
    'MEDIUM': 1.0,
    'HARD': 0.6,
    'INTERMEDIATE': 0.8,
    'WET': 0.7
}

# Energy weighting factors (normalized to balance contributions)
K_BRAKING = 0.001        # Braking energy coefficient
K_CORNERING = 0.02       # Cornering energy coefficient  
K_ACCELERATION = 0.0005  # Acceleration slip coefficient
K_THERMAL = 0.01         # Thermal load coefficient

# Temperature effect parameters
ALPHA_TEMP = 0.5         # Temperature exponential factor

# Degradation parameters
BETA_DECAY = 0.03        # Degradation accumulation rate
GAMMA_LAPTIME = 0.10     # Lap time sensitivity (10% slower when fully worn)

print("Mathematical model parameters loaded")
print(f"\nCompound coefficients: {COMPOUND_COEFFICIENTS}")
print(f"Lap time sensitivity: {GAMMA_LAPTIME*100:.1f}% on fully worn tires")

In [ ]:
def calculate_braking_energy(row):
    """
    W_braking = K * m * v_mean * brake_applications * brake_intensity
    
    This approximates kinetic energy dissipated during braking events.
    """
    v_mean_ms = row['speed_mean'] / 3.6  # Convert km/h to m/s
    brake_intensity = row['brake_mean'] / 100.0  # Normalize to 0-1
    
    energy = K_BRAKING * VEHICLE_MASS * v_mean_ms * row['brake_applications'] * (brake_intensity + 0.1)
    return energy

def calculate_cornering_energy(row):
    """
    W_cornering = K * speed_std^2 * telemetry_points
    
    Speed variance indicates cornering (lateral acceleration), which causes tire slip.
    """
    energy = K_CORNERING * (row['speed_std'] ** 2) * row['telemetry_points']
    return energy

def calculate_acceleration_energy(row):
    """
    W_acceleration = K * throttle_mean * speed_mean * lap_duration
    
    High throttle at speed causes tire slip and wear.
    """
    v_mean_ms = row['speed_mean'] / 3.6
    throttle_norm = row['throttle_mean'] / 100.0
    
    energy = K_ACCELERATION * throttle_norm * v_mean_ms * row['lap_duration']
    return energy

def calculate_thermal_load(row):
    """
    W_thermal = K * (speed_mean * lap_duration + gear_shifts * 10)
    
    Sustained high speeds and gear shifts generate heat, increasing wear.
    """
    thermal = K_THERMAL * (row['speed_mean'] * row['lap_duration'] + row['gear_shifts'] * 10)
    return thermal

def calculate_temperature_factor(row, max_speed, max_brake_apps):
    """
    T_factor = exp(alpha * T_normalized)
    
    Temperature proxy based on:
    - Normalized average speed (high speed = high tire temperature)
    - Brake applications (braking heats tires)
    - Throttle usage (acceleration heats tires)
    """
    speed_norm = row['speed_mean'] / max_speed if max_speed > 0 else 0
    brake_norm = row['brake_applications'] / max_brake_apps if max_brake_apps > 0 else 0
    throttle_norm = row['throttle_mean'] / 100.0
    
    T_normalized = 0.4 * speed_norm + 0.3 * brake_norm + 0.3 * throttle_norm
    
    T_factor = np.exp(ALPHA_TEMP * T_normalized)
    return T_factor

def calculate_lap_wear(row, max_speed, max_brake_apps):
    """
    Total wear for a single lap:
    W_lap = k_compound * (W_braking + W_cornering + W_acceleration + W_thermal) * T_factor
    """
    # Energy components
    W_brake = calculate_braking_energy(row)
    W_corner = calculate_cornering_energy(row)
    W_accel = calculate_acceleration_energy(row)
    W_thermal = calculate_thermal_load(row)
    
    # Total mechanical energy
    W_mechanical = W_brake + W_corner + W_accel + W_thermal
    
    # Compound coefficient
    k_compound = COMPOUND_COEFFICIENTS.get(row['compound'], 1.0)
    
    # Temperature multiplier
    T_factor = calculate_temperature_factor(row, max_speed, max_brake_apps)
    
    # Total lap wear
    W_lap = k_compound * W_mechanical * T_factor
    
    return W_lap, W_brake, W_corner, W_accel, W_thermal, T_factor

print("Mathematical formulas implemented")

## 7. Calculate Tire Wear for Each Lap

In [ ]:
# Calculate normalization factors
max_speed = df['speed_mean'].max()
max_brake_apps = df['brake_applications'].max()

print(f"Normalization factors:")
print(f"  Max average speed: {max_speed:.1f} km/h")
print(f"  Max brake applications: {max_brake_apps}")
print("\nCalculating wear for each lap...")

# Apply wear calculation to each lap
wear_results = df.apply(lambda row: calculate_lap_wear(row, max_speed, max_brake_apps), axis=1)

# Unpack results
df['wear_total'] = [w[0] for w in wear_results]
df['wear_braking'] = [w[1] for w in wear_results]
df['wear_cornering'] = [w[2] for w in wear_results]
df['wear_acceleration'] = [w[3] for w in wear_results]
df['wear_thermal'] = [w[4] for w in wear_results]
df['temp_factor'] = [w[5] for w in wear_results]

print(f"\nWear calculated for {len(df)} laps")
print("\nWear statistics:")
print(df[['wear_total', 'wear_braking', 'wear_cornering', 'wear_acceleration', 'wear_thermal']].describe())

## 8. Calculate Cumulative Degradation

We need to track degradation within each tire stint separately.

In [ ]:
def calculate_cumulative_degradation(df):
    """
    Calculate cumulative degradation for each stint.
    
    D_cumulative(n) = sum of all wear up to lap n within the stint
    D_normalized(n) = 1 - exp(-beta * D_cumulative)
    """
    df = df.sort_values('lap_number').copy()
    
    # Detect tire stints (when compound changes or tyre_life resets)
    df['stint_id'] = ((df['compound'] != df['compound'].shift()) | 
                      (df['tyre_life'] < df['tyre_life'].shift())).cumsum()
    
    # Calculate cumulative wear within each stint
    df['wear_cumulative'] = df.groupby('stint_id')['wear_total'].cumsum()
    
    # Normalized degradation state (0 = new, 1 = fully worn)
    df['degradation_normalized'] = 1 - np.exp(-BETA_DECAY * df['wear_cumulative'])
    
    return df

df = calculate_cumulative_degradation(df)

print("Cumulative degradation calculated")
print(f"\nNumber of stints: {df['stint_id'].nunique()}")
print("\nStint summary:")
stint_summary = df.groupby(['stint_id', 'compound']).agg({
    'lap_number': ['first', 'last', 'count'],
    'degradation_normalized': ['min', 'max']
})
print(stint_summary)

## 9. Calculate Predicted Lap Times

Using the degradation state, predict lap time impact.

In [ ]:
def calculate_predicted_lap_times(df):
    """
    Calculate predicted lap time based on tire degradation:
    
    t_lap(n) = t_baseline * (1 + gamma * D_normalized(n))
    
    where t_baseline is the best lap time for each stint (assumed to be first lap on fresh tires)
    """
    df = df.copy()
    
    # For each stint, find baseline (best lap time, typically early in stint)
    # We'll use the fastest lap in first 3 laps of each stint as baseline
    def get_baseline_time(stint_df):
        early_laps = stint_df.head(3)
        return early_laps['lap_time_seconds'].min()
    
    df['baseline_time'] = df.groupby('stint_id')['lap_time_seconds'].transform(get_baseline_time)
    
    # Predicted lap time based on degradation
    df['predicted_lap_time'] = df['baseline_time'] * (1 + GAMMA_LAPTIME * df['degradation_normalized'])
    
    # Residual: actual - predicted
    df['lap_time_residual'] = df['lap_time_seconds'] - df['predicted_lap_time']
    
    # Time loss due to tire wear
    df['time_loss_wear'] = df['predicted_lap_time'] - df['baseline_time']
    
    return df

df = calculate_predicted_lap_times(df)

print("Predicted lap times calculated")
print("\nModel performance metrics:")

# Calculate RMSE and R²
rmse = np.sqrt(np.mean(df['lap_time_residual'] ** 2))
ss_res = np.sum(df['lap_time_residual'] ** 2)
ss_tot = np.sum((df['lap_time_seconds'] - df['lap_time_seconds'].mean()) ** 2)
r_squared = 1 - (ss_res / ss_tot)

print(f"  RMSE: {rmse:.3f} seconds")
print(f"  R²: {r_squared:.3f}")
print(f"  Mean absolute error: {df['lap_time_residual'].abs().mean():.3f} seconds")

# Show sample predictions
print("\nSample predictions:")
display_cols = ['lap_number', 'compound', 'tyre_life', 'lap_time_seconds', 
                'predicted_lap_time', 'lap_time_residual', 'degradation_normalized']
df[display_cols].head(10)

## 10. Wear Component Analysis

Analyze which factors contribute most to tire wear.

In [ ]:
# Calculate percentage contribution of each wear component
wear_components = ['wear_braking', 'wear_cornering', 'wear_acceleration', 'wear_thermal']
wear_totals = df[wear_components].sum()
wear_percentages = (wear_totals / wear_totals.sum() * 100).sort_values(ascending=False)

print("Tire Wear Component Analysis")
print("="*50)
for component, percentage in wear_percentages.items():
    clean_name = component.replace('wear_', '').title()
    print(f"{clean_name:15s}: {percentage:5.1f}%")

# Visualize wear components
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#ffa07a']
labels = [name.replace('wear_', '').title() for name in wear_percentages.index]
ax1.pie(wear_percentages.values, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
ax1.set_title('Tire Wear Component Distribution', fontsize=14, fontweight='bold')

# Bar chart by stint
stint_wear = df.groupby('stint_id')[wear_components].sum()
stint_wear.plot(kind='bar', stacked=True, ax=ax2, color=colors)
ax2.set_title('Wear Components by Stint', fontsize=14, fontweight='bold')
ax2.set_xlabel('Stint ID')
ax2.set_ylabel('Cumulative Wear')
ax2.legend([label.replace('wear_', '').title() for label in wear_components], loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('wear_component_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved as 'wear_component_analysis.png'")

## 11. Simulate Improved Tire Scenario

Modify wear coefficients to simulate a tire with better durability.

In [ ]:
def simulate_improved_tire(df_original, improvement_factor=0.30):
    """
    Simulate improved tire by reducing wear coefficient.
    
    improvement_factor: percentage reduction in wear rate (e.g., 0.30 = 30% less wear)
    
    Returns DataFrame with improved tire predictions.
    """
    df_improved = df_original.copy()
    
    print(f"Simulating tire with {improvement_factor*100:.0f}% improvement (reduced wear rate)")
    
    # Reduce total wear by improvement factor
    df_improved['wear_total_improved'] = df_improved['wear_total'] * (1 - improvement_factor)
    
    # Recalculate cumulative degradation with improved wear
    df_improved = df_improved.sort_values('lap_number')
    df_improved['wear_cumulative_improved'] = df_improved.groupby('stint_id')['wear_total_improved'].cumsum()
    df_improved['degradation_normalized_improved'] = 1 - np.exp(-BETA_DECAY * df_improved['wear_cumulative_improved'])
    
    # Recalculate predicted lap times with improved tire
    df_improved['predicted_lap_time_improved'] = df_improved['baseline_time'] * (
        1 + GAMMA_LAPTIME * df_improved['degradation_normalized_improved']
    )
    
    # Time savings
    df_improved['time_saved_per_lap'] = df_improved['predicted_lap_time'] - df_improved['predicted_lap_time_improved']
    
    return df_improved

# Simulate multiple improvement scenarios
IMPROVEMENT_SCENARIOS = [0.15, 0.30, 0.50]  # 15%, 30%, 50% improvement

improved_scenarios = {}
for improvement in IMPROVEMENT_SCENARIOS:
    improved_scenarios[improvement] = simulate_improved_tire(df, improvement)

print("\nImprovement scenarios calculated")
print(f"Baseline tire: Standard compound")
for improvement in IMPROVEMENT_SCENARIOS:
    print(f"  Scenario {improvement*100:.0f}%: {improvement*100:.0f}% reduced wear rate")

## 12. Calculate Performance Benefits

In [ ]:
print("Performance Analysis: Improved Tire Scenarios")
print("="*70)

for improvement, df_scenario in improved_scenarios.items():
    print(f"\n{improvement*100:.0f}% Improvement Scenario:")
    print("-"*70)
    
    # Total time saved
    total_time_saved = df_scenario['time_saved_per_lap'].sum()
    print(f"  Total time saved over race: {total_time_saved:.2f} seconds")
    
    # Average time saved per lap
    avg_time_saved = df_scenario['time_saved_per_lap'].mean()
    print(f"  Average time saved per lap: {avg_time_saved:.3f} seconds")
    
    # By stint
    print("\n  Time saved by stint:")
    stint_savings = df_scenario.groupby(['stint_id', 'compound']).agg({
        'time_saved_per_lap': 'sum',
        'lap_number': 'count'
    }).rename(columns={'time_saved_per_lap': 'total_saved', 'lap_number': 'laps'})
    
    for idx, row in stint_savings.iterrows():
        stint_id, compound = idx
        print(f"    Stint {stint_id} ({compound}): {row['total_saved']:.2f}s over {row['laps']} laps")
    
    # Maximum instantaneous benefit
    max_benefit_lap = df_scenario.loc[df_scenario['time_saved_per_lap'].idxmax()]
    print(f"\n  Maximum benefit: {max_benefit_lap['time_saved_per_lap']:.3f}s on lap {max_benefit_lap['lap_number']:.0f}")

# Create summary table
summary_data = []
for improvement, df_scenario in improved_scenarios.items():
    summary_data.append({
        'Improvement': f"{improvement*100:.0f}%",
        'Total Time Saved (s)': df_scenario['time_saved_per_lap'].sum(),
        'Avg Per Lap (s)': df_scenario['time_saved_per_lap'].mean(),
        'Max Single Lap (s)': df_scenario['time_saved_per_lap'].max(),
        'Final Deg. State': df_scenario['degradation_normalized_improved'].iloc[-1]
    })

summary_df = pd.DataFrame(summary_data)
print("\n\nScenario Comparison Summary:")
print("="*70)
print(summary_df.to_string(index=False))

## 13. Visualizations

In [ ]:
# Main comparison plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle(f'Tire Wear Analysis - {DRIVER} - {YEAR} {RACE} GP', 
             fontsize=16, fontweight='bold', y=0.995)

# Plot 1: Lap time degradation
ax1 = axes[0, 0]
ax1.scatter(df['lap_number'], df['lap_time_seconds'], alpha=0.6, label='Actual', s=50)
ax1.plot(df['lap_number'], df['predicted_lap_time'], 'r-', label='Baseline Predicted', linewidth=2)

# Plot improved scenarios
colors_improved = ['green', 'blue', 'purple']
for i, (improvement, df_scenario) in enumerate(improved_scenarios.items()):
    ax1.plot(df_scenario['lap_number'], df_scenario['predicted_lap_time_improved'], 
             linestyle='--', linewidth=2, color=colors_improved[i],
             label=f'{improvement*100:.0f}% Improved')

ax1.set_xlabel('Lap Number', fontweight='bold')
ax1.set_ylabel('Lap Time (seconds)', fontweight='bold')
ax1.set_title('Lap Time vs Tire Degradation', fontweight='bold')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Add compound color bands
for stint_id in df['stint_id'].unique():
    stint_data = df[df['stint_id'] == stint_id]
    compound = stint_data['compound'].iloc[0]
    ax1.axvspan(stint_data['lap_number'].min(), stint_data['lap_number'].max(), 
                alpha=0.1, color='red' if compound == 'SOFT' else 'yellow' if compound == 'MEDIUM' else 'white')

# Plot 2: Degradation state over time
ax2 = axes[0, 1]
ax2.plot(df['lap_number'], df['degradation_normalized'], 'r-', linewidth=2, label='Baseline')

for i, (improvement, df_scenario) in enumerate(improved_scenarios.items()):
    ax2.plot(df_scenario['lap_number'], df_scenario['degradation_normalized_improved'],
             linestyle='--', linewidth=2, color=colors_improved[i],
             label=f'{improvement*100:.0f}% Improved')

ax2.set_xlabel('Lap Number', fontweight='bold')
ax2.set_ylabel('Normalized Degradation State', fontweight='bold')
ax2.set_title('Tire Degradation State (0=new, 1=worn)', fontweight='bold')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1])

# Plot 3: Time saved per lap
ax3 = axes[1, 0]
for i, (improvement, df_scenario) in enumerate(improved_scenarios.items()):
    ax3.plot(df_scenario['lap_number'], df_scenario['time_saved_per_lap'],
             linewidth=2, color=colors_improved[i], marker='o', markersize=4,
             label=f'{improvement*100:.0f}% Improved')

ax3.set_xlabel('Lap Number', fontweight='bold')
ax3.set_ylabel('Time Saved (seconds)', fontweight='bold')
ax3.set_title('Time Saved Per Lap (vs Baseline)', fontweight='bold')
ax3.legend(loc='best')
ax3.grid(True, alpha=0.3)
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.8)

# Plot 4: Cumulative time saved
ax4 = axes[1, 1]
for i, (improvement, df_scenario) in enumerate(improved_scenarios.items()):
    cumulative_saved = df_scenario['time_saved_per_lap'].cumsum()
    ax4.plot(df_scenario['lap_number'], cumulative_saved,
             linewidth=3, color=colors_improved[i],
             label=f'{improvement*100:.0f}% Improved')
    
    # Add final value annotation
    final_val = cumulative_saved.iloc[-1]
    ax4.annotate(f'{final_val:.1f}s', 
                xy=(df_scenario['lap_number'].iloc[-1], final_val),
                xytext=(10, 0), textcoords='offset points',
                fontsize=10, fontweight='bold', color=colors_improved[i])

ax4.set_xlabel('Lap Number', fontweight='bold')
ax4.set_ylabel('Cumulative Time Saved (seconds)', fontweight='bold')
ax4.set_title('Cumulative Time Advantage', fontweight='bold')
ax4.legend(loc='best')
ax4.grid(True, alpha=0.3)
ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.8)

plt.tight_layout()
plt.savefig('tire_wear_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nMain visualization saved as 'tire_wear_comparison.png'")

## 14. Additional Analysis: Stint-by-Stint Comparison

In [ ]:
# Create detailed stint analysis
num_stints = df['stint_id'].nunique()

fig, axes = plt.subplots(num_stints, 1, figsize=(14, 5*num_stints))
if num_stints == 1:
    axes = [axes]

for stint_idx, stint_id in enumerate(sorted(df['stint_id'].unique())):
    ax = axes[stint_idx]
    
    # Filter data for this stint
    stint_df = df[df['stint_id'] == stint_id]
    compound = stint_df['compound'].iloc[0]
    
    # Plot actual and predicted
    ax.scatter(stint_df['tyre_life'], stint_df['lap_time_seconds'], 
              alpha=0.6, s=80, label='Actual', zorder=3)
    ax.plot(stint_df['tyre_life'], stint_df['predicted_lap_time'], 
           'r-', linewidth=2.5, label='Baseline Model', zorder=2)
    
    # Plot improved scenarios
    for i, (improvement, df_scenario) in enumerate(improved_scenarios.items()):
        stint_improved = df_scenario[df_scenario['stint_id'] == stint_id]
        ax.plot(stint_improved['tyre_life'], stint_improved['predicted_lap_time_improved'],
               linestyle='--', linewidth=2, color=colors_improved[i],
               label=f'{improvement*100:.0f}% Improved', zorder=2)
    
    ax.set_xlabel('Tire Age (laps)', fontweight='bold')
    ax.set_ylabel('Lap Time (seconds)', fontweight='bold')
    ax.set_title(f'Stint {stint_id} - {compound} Compound', fontweight='bold', fontsize=12)
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('stint_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Stint-by-stint analysis saved as 'stint_analysis.png'")

## 15. Model Formula Summary and Documentation

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════════════════╗
║             MATHEMATICAL TIRE WEAR MODEL - FORMULA SUMMARY                 ║
╚════════════════════════════════════════════════════════════════════════════╝

1. ENERGY DISSIPATION COMPONENTS
   ────────────────────────────────
   
   Braking Energy:
     W_brake = K_brake × m × v_mean × N_brake × brake_intensity
     
   Cornering Energy:
     W_corner = K_corner × σ_speed² × N_telemetry
     
   Acceleration Energy:
     W_accel = K_accel × throttle_norm × v_mean × t_lap
     
   Thermal Load:
     W_thermal = K_thermal × (v_mean × t_lap + N_shifts × 10)

2. TEMPERATURE FACTOR
   ──────────────────
   
   T_normalized = 0.4×(v/v_max) + 0.3×(N_brake/N_max) + 0.3×(throttle/100)
   T_factor = exp(α × T_normalized)
   
   where α = {ALPHA_TEMP}

3. TOTAL LAP WEAR
   ───────────────
   
   W_lap = k_compound × (W_brake + W_corner + W_accel + W_thermal) × T_factor
   
   Compound coefficients:
""")

for compound, coef in COMPOUND_COEFFICIENTS.items():
    print(f"     {compound:12s}: k = {coef:.2f}")

print(f"""
4. CUMULATIVE DEGRADATION
   ───────────────────────
   
   D_cumulative(n) = Σ W_lap(i) for i=1 to n
   
   D_normalized(n) = 1 - exp(-β × D_cumulative)
   
   where β = {BETA_DECAY}

5. LAP TIME IMPACT
   ────────────────
   
   t_predicted(n) = t_baseline × (1 + γ × D_normalized(n))
   
   where γ = {GAMMA_LAPTIME} (meaning {GAMMA_LAPTIME*100:.0f}% slower when fully worn)

6. IMPROVED TIRE SIMULATION
   ─────────────────────────
   
   W_lap_improved = W_lap × (1 - improvement_factor)
   
   Then recalculate D_cumulative and t_predicted with reduced wear.

╔════════════════════════════════════════════════════════════════════════════╗
║                          MODEL COEFFICIENTS USED                           ║
╚════════════════════════════════════════════════════════════════════════════╝

Vehicle mass (m):         {VEHICLE_MASS} kg
Braking coefficient:      {K_BRAKING}
Cornering coefficient:    {K_CORNERING}
Acceleration coefficient: {K_ACCELERATION}
Thermal coefficient:      {K_THERMAL}
Temperature exponent (α): {ALPHA_TEMP}
Decay rate (β):           {BETA_DECAY}
Lap time sensitivity (γ): {GAMMA_LAPTIME}

╔════════════════════════════════════════════════════════════════════════════╗
║                            KEY ASSUMPTIONS                                 ║
╚════════════════════════════════════════════════════════════════════════════╝

1. Tire wear is primarily driven by mechanical energy dissipation
2. Temperature effects are approximated from telemetry (speed, braking, throttle)
3. Each compound has a characteristic wear rate coefficient
4. Degradation accumulates exponentially (not linearly) within a stint
5. Lap time increases proportionally to normalized degradation state
6. Baseline lap time for each stint is the fastest lap in first 3 laps
7. Model does not account for:
   - Track evolution / rubber buildup
   - Fuel load changes
   - Weather/track temperature variations
   - Traffic / racing conditions
   - Driver strategy changes

╔════════════════════════════════════════════════════════════════════════════╗
║                         LIMITATIONS & CAVEATS                              ║
╚════════════════════════════════════════════════════════════════════════════╝

• This is a MATHEMATICAL MODEL, not AI/ML - all relationships are formula-based
• Coefficients are engineering estimates; real F1 teams use proprietary data
• Brake telemetry in FastF1 may be limited (binary on/off rather than pressure)
• Model is calibrated per-driver, per-race; results vary by track characteristics
• "Improved tire" simulation is theoretical; actual tire development involves
  complex chemistry, structure, and compound trade-offs
• Residuals may be affected by non-wear factors (traffic, mistakes, strategy)

""")

## 16. Export Results

In [ ]:
# Export detailed results to CSV
output_filename = f'tire_wear_results_{DRIVER}_{YEAR}_{RACE}.csv'

# Combine baseline and improved scenarios into one export
export_df = df.copy()

for improvement, df_scenario in improved_scenarios.items():
    suffix = f'_improved_{int(improvement*100)}pct'
    export_df[f'predicted_lap_time{suffix}'] = df_scenario['predicted_lap_time_improved']
    export_df[f'time_saved{suffix}'] = df_scenario['time_saved_per_lap']

# Select key columns for export
export_cols = [
    'lap_number', 'stint_id', 'compound', 'tyre_life',
    'lap_time_seconds', 'baseline_time', 'predicted_lap_time',
    'wear_total', 'wear_braking', 'wear_cornering', 'wear_acceleration', 'wear_thermal',
    'degradation_normalized', 'temp_factor',
    'predicted_lap_time_improved_15pct', 'time_saved_improved_15pct',
    'predicted_lap_time_improved_30pct', 'time_saved_improved_30pct',
    'predicted_lap_time_improved_50pct', 'time_saved_improved_50pct'
]

export_df[export_cols].to_csv(output_filename, index=False)
print(f"Results exported to '{output_filename}'")

# Create summary report
summary_filename = f'tire_wear_summary_{DRIVER}_{YEAR}_{RACE}.txt'

with open(summary_filename, 'w') as f:
    f.write("="*80 + "\n")
    f.write(f"  TIRE WEAR MATHEMATICAL MODEL - RESULTS SUMMARY\n")
    f.write(f"  {YEAR} {RACE} Grand Prix - Driver: {DRIVER}\n")
    f.write("="*80 + "\n\n")
    
    f.write(f"Model Performance:\n")
    f.write(f"  RMSE: {rmse:.3f} seconds\n")
    f.write(f"  R²: {r_squared:.3f}\n")
    f.write(f"  Mean Absolute Error: {df['lap_time_residual'].abs().mean():.3f} seconds\n\n")
    
    f.write(f"Wear Component Breakdown:\n")
    for component, percentage in wear_percentages.items():
        clean_name = component.replace('wear_', '').title()
        f.write(f"  {clean_name:15s}: {percentage:5.1f}%\n")
    f.write("\n")
    
    f.write(f"Improved Tire Scenario Results:\n")
    f.write(f"{'-'*80}\n")
    for improvement, df_scenario in improved_scenarios.items():
        total_saved = df_scenario['time_saved_per_lap'].sum()
        avg_saved = df_scenario['time_saved_per_lap'].mean()
        f.write(f"\n{improvement*100:.0f}% Improvement:\n")
        f.write(f"  Total time saved: {total_saved:.2f} seconds\n")
        f.write(f"  Average per lap: {avg_saved:.3f} seconds\n")
        f.write(f"  Maximum single lap: {df_scenario['time_saved_per_lap'].max():.3f} seconds\n")

print(f"Summary report saved to '{summary_filename}'")
print("\nAll analysis complete!")

## 17. Conclusions and Next Steps

### Key Findings:

1. **Mathematical Model Performance**: This physics-based formula approach provides transparent, interpretable predictions of tire wear and lap time degradation

2. **Wear Contributors**: Analysis shows the relative contribution of braking, cornering, acceleration, and thermal effects to overall tire wear

3. **Improved Tire Benefits**: Simulations demonstrate quantifiable time savings from reduced wear rates

### Model Strengths:
- No AI/ML "black box" - all calculations are explicit formulas
- Based on physical principles of energy dissipation
- Transparent assumptions and coefficients
- Easy to modify and tune for different scenarios

### Limitations:
- Simplified physics (real tire behavior is more complex)
- Coefficients are estimates, not lab-tested values
- Doesn't capture all real-world factors (track evolution, weather, etc.)
- Limited by available telemetry data

### Potential Improvements:
1. **Calibration**: Tune coefficients using more extensive historical data
2. **Track-Specific Models**: Different tracks have different wear characteristics
3. **Weather Integration**: Add temperature and rain effects
4. **Fuel Load**: Account for decreasing weight over race
5. **Lateral G-Forces**: If available, use actual lateral acceleration data for cornering wear
6. **Multi-Driver Comparison**: Compare wear patterns across drivers

### Ethical Considerations:
- Data sourced from public FastF1 library (FIA-provided telemetry)
- Model assumptions are transparent and documented
- Results are educational/analytical, not competitive intelligence
- Driver performance analysis respects privacy (aggregated data)

### Christian Worldview Integration:
- **Stewardship**: Better tire management reduces waste and resource consumption
- **Safety**: Understanding tire degradation improves safety margins
- **Fairness**: Transparent formulas ensure equitable analysis
- **Excellence**: Pursuing technical accuracy honors God-given abilities
- **Service**: Using engineering skills to benefit team performance